## HOMEWORK No 11 : OpenAI API Overview

Write prompts using OPENAI API keys that perform the following task and display the output:

1. Write a chatbot prompt to iteratively create a sequence of chats on one particular topic.

      a. Ask the bot to solve one complex math problem.

      b. Give a PDF and website document; ask the bot to rewrite and answer questions on the given PDF and website.

      c. At the end, ask the bot to summarize your chat.

In [1]:
import openai
import os
from getpass import getpass
import pdfplumber
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### Setting up the environment with open AI API_Key

In [2]:
def get_api_key():

    api_key = os.getenv('OPENAI_API_KEY')

    if not api_key:
        api_key = getpass("Please enter your OpenAI API key: ").strip()
        
        save_choice = input("Would you like to save this API key for future use? (yes/no): ").lower()
        if save_choice == 'yes':
            os.environ['OPENAI_API_KEY'] = api_key
            print("API key saved as an environment variable.")
    
    return api_key

#### Function to interact with OpenAI chat completion API

In [3]:
def chat_with_gpt(conversation):
    openai.api_key = get_api_key()
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=conversation,
            max_tokens=2000
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

#### Initialize conversation messages

##### The System Role is to define the bot's role as an assistant that solves complex math problems.And User Role is to provide the specific math problem to solve.

In [4]:
conversation = [
        {"role": "system", "content": "You are a helpful assistant designed to engage in a multi-step conversation."},
        {"role": "user", "content": "Hello, ChatGPT! Can you briefly introduce yourself?"}
    ]

In [5]:
response = chat_with_gpt(conversation)
print("Chatbot Response:", response)

Please enter your OpenAI API key:  ········
Would you like to save this API key for future use? (yes/no):  yes


API key saved as an environment variable.
Chatbot Response: Hello! I'm an AI assistant here to help you with your questions and have engaging conversations on a wide range of topics. Feel free to ask me anything, and I'll do my best to assist you! What would you like to talk about today?


#### Appending the conversation to maintain the chat conversation history

In [6]:
conversation.append({"role": "assistant", "content": response})

In [7]:
conversation

[{'role': 'system',
  'content': 'You are a helpful assistant designed to engage in a multi-step conversation.'},
 {'role': 'user',
  'content': 'Hello, ChatGPT! Can you briefly introduce yourself?'},
 {'role': 'assistant',
  'content': "Hello! I'm an AI assistant here to help you with your questions and have engaging conversations on a wide range of topics. Feel free to ask me anything, and I'll do my best to assist you! What would you like to talk about today?"}]

## a. Ask the bot to solve one complex math problem.

In [8]:
conversation.append(       
        {"role": "user", "content": "Please solve this complex math problem step by step: Calculate the surface area and volume of a sphere with a radius of 7.3 meters. Show all your work and explain each calculation."}
)

In [9]:
math_solution = chat_with_gpt(conversation)
print("\n--- Complex Math Problem Solution ---")
print(math_solution)


--- Complex Math Problem Solution ---
To find the surface area and volume of a sphere, we use the following formulas:

1. Surface area of a sphere: A = 4πr²
2. Volume of a sphere: V = 4/3πr³

Given that the radius (r) is 7.3 meters, we can substitute this value into the formulas and solve step by step.

1. Surface Area (A) of the sphere:
A = 4π(7.3)²
A = 4π(53.29)  [Calculating the square of the radius]
A ≈ 212.39π square meters
A ≈ 668.06 square meters

Therefore, the surface area of the sphere is approximately 668.06 square meters.

2. Volume (V) of the sphere:
V = 4/3π(7.3)³
V = 4/3π(389.017) [Calculating the cube of the radius]
V ≈ 1631.65π cubic meters
V ≈ 5125.61 cubic meters

Therefore, the volume of the sphere is approximately 5125.61 cubic meters.

So, for a sphere with a radius of 7.3 meters, the surface area is approximately 668.06 square meters and the volume is approximately 5125.61 cubic meters.


In [10]:
conversation.append({"role": "assistant", "content": math_solution})

## b. Give a PDF and website document; ask the bot to rewrite and answer questions on the given PDF and website.

#### Function to extract text from pdf on integers

In [11]:
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

In [12]:
pdf_text = extract_text_from_pdf('Integers.pdf')

In [13]:
len(pdf_text)

3316

#### Function for extracting text from website containing content on maths operations details

In [14]:
def extract_text_from_website(url):
    """Extract text content from a webpage."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Extract all paragraph text
    text = " ".join([p.get_text() for p in soup.find_all('p')])
    return text

In [15]:
website_text = extract_text_from_website("https://www.ncl.ac.uk/webtemplate/ask-assets/external/maths-resources/numeracy/order-of-operations-bodmas.html")

In [16]:
len(website_text)

2550

#### Combining both the content

In [17]:
combined_content = f"PDF Content:\n{pdf_text}\n\n WebSite Content:{website_text}"

In [18]:
len(combined_content)

5898

In [19]:
audience="8th-grade"
content = combined_content
conversation.append({"role": "user", "content": f"I am providing an extract from a website and a pdf for reference"})
conversation.append({"role": "user", "content": f"Rewrite the content for a {audience} audience while retaining all key concepts and provide some examples: {content}"})

#### Summarizing the content from both sources

In [20]:
rewritten_content = chat_with_gpt(conversation)
print("\n--- Rewritten Content ---")
print(rewritten_content)


--- Rewritten Content ---
Sure, here is a simplified explanation of the concepts discussed in the PDF document:

**Integers:**
Integers are positive and negative whole numbers. When you perform math operations on integers, you follow the order of operations, which includes addition, subtraction, multiplication, and division.

**Order of Operations:**
1. First, do operations inside grouping symbols like parentheses.
Example: \(3(12 - 5) = 3(7) = 21\)

2. Next, evaluate exponents and roots before other operations.
Example: \(12 - 3^2 = 12 - 9 = 3\)

3. Then, perform multiplication and division before addition and subtraction.
Example: \(4 + 20 ÷ 2 * 5 = 4 + 10 * 5 = 4 + 50 = 54\)

4. Lastly, do addition and subtraction. You can change the order to make it easier.
Example: \(53 + 14 + 7 = 53 + 7 + 14 = 60 + 14 = 74\)

**Rules of Integers:**
1. Adding two integers:
   - When the signs are the same, add and keep the sign.
   - When the signs are different, subtract and keep the sign of the

In [21]:
conversation.append({"role": "assistant", "content": rewritten_content})

#### Questions and Answers

In [22]:
questions = [
        "What happens when you multiple two negatives?",
        "In what orders are the mathematical operations performed?",
        "When you multiply integers what determines the results?.",
        "What is the answer for the following, provide step by step solution, (8*9)-8/4+6*5 ?",
        "What is the difference between PEMDAS and BODMAS?",
        "Which rule is most commonly used in real world scenarios, PEMDAS or BODMAS?"
    ]

conversation.append({"role": "user", 
                     "content": f"The following is the content for reference: {rewritten_content}"})

answers = []
for question in questions:
    conversation.append({"role": "user", "content": question})
    answer = chat_with_gpt(conversation)
    answers.append(answer)
    conversation.append({"role": "assistant", "content": answer})

print(answers)

['When you multiply two negative numbers together, the result is always a positive number. This rule applies to integers and is a fundamental property of multiplication.\n\nFor example:\n(-3) x (-4) = 12\n(-2) x (-6) = 12\n\nSo, when you multiply two negative numbers, the product will be a positive number.', 'The mathematical operations are performed following the order of operations, also known as "PEMDAS" or "BODMAS." The acronym stands for:\n\n1. Parentheses (or Brackets)\n2. Exponents (or Orders)\n3. Multiplication and Division (from left to right)\n4. Addition and Subtraction (from left to right)\n\nThis order ensures that calculations are done consistently and accurately. By following the order of operations, you can avoid confusion and arrive at the correct answer for mathematical expressions.', 'When you multiply integers, the signs of the numbers being multiplied together determine the result. Here are the rules for multiplying integers:\n\n1. **Two positives multiply to a pos

In [23]:
for i, answer in enumerate(answers):
    print(f"Q: {questions[i]}\nA: {answer}\n")

Q: What happens when you multiple two negatives?
A: When you multiply two negative numbers together, the result is always a positive number. This rule applies to integers and is a fundamental property of multiplication.

For example:
(-3) x (-4) = 12
(-2) x (-6) = 12

So, when you multiply two negative numbers, the product will be a positive number.

Q: In what orders are the mathematical operations performed?
A: The mathematical operations are performed following the order of operations, also known as "PEMDAS" or "BODMAS." The acronym stands for:

1. Parentheses (or Brackets)
2. Exponents (or Orders)
3. Multiplication and Division (from left to right)
4. Addition and Subtraction (from left to right)

This order ensures that calculations are done consistently and accurately. By following the order of operations, you can avoid confusion and arrive at the correct answer for mathematical expressions.

Q: When you multiply integers what determines the results?.
A: When you multiply integer

## Ask Bot to Summarize the chat

In [24]:
conversation.append({
        "role": "user", 
        "content": "Please provide a comprehensive summary of our entire conversation, highlighting the key points discussed, the maths problem solved, the problem-solving approach, and the main insights gained."
    })

### Summarized Chat History

In [25]:
chat_summary = chat_with_gpt(conversation)
print("\n--- Conversation Summary ---")
print(chat_summary)


--- Conversation Summary ---
During our conversation, we covered various mathematical concepts related to integers, order of operations, and multiplication rules. Here is a summary of our discussion:

**Key Points Discussed:**
1. **Integers:** Integers are positive and negative whole numbers.
2. **Order of Operations (PEMDAS/BODMAS):** The order of operations guides the sequence in which mathematical calculations are performed: Parentheses, Exponents, Multiplication, Division, Addition, and Subtraction.
3. **Multiplying Integers:** When multiplying integers, the rules determine the sign of the product: two positives result in a positive, two negatives result in a positive, and a positive and a negative result in a negative.
4. **PEMDAS vs. BODMAS:** Both acronyms represent the same order of operations, with a slight variation in the order of multiplication and division.
5. **Real-World Application:** The PEMDAS rule is more commonly used in real-world scenarios and mathematical practi

#### Appending this chat History in the conversation

In [26]:
conversation.append({
        "role": "assistant", 
        "content": chat_summary
    })

#### Visualizing the Chat History in a Pandas Dataframe

In [27]:

df = pd.DataFrame(conversation)

# Set pandas display options to show full content of each column
pd.set_option('display.max_colwidth', None)  # Set no truncation for column content
pd.set_option('display.max_rows', None)  # Optional: to show all rows if the DataFrame is large

# Print the DataFrame in a tabular format
display(df)

role  \
0      system   
1        user   
2   assistant   
3        user   
4   assistant   
5        user   
6        user   
7   assistant   
8        user   
9        user   
10  assistant   
11       user   
12  assistant   
13       user   
14  assistant   
15       user   
16  assistant   
17       user   
18  assistant   
19       user   
20  assistant   
21       user   
22  assistant   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## Thank You